In [42]:
import pandas as pd
import os

### rotten tomatoes

In [43]:
# DIR_NAME = "./test_cases_rotten_tomatoes_rotulated_NO"
# DIR_NAME = "./test_cases_rotten_tomatoes_adson"
DIR_NAME = "../notebooks_amazon/test_cases_amazon_rotulated/test_cases_amazon_rotulated_NO"
# DIR_NAME = "../notebooks_imdb/test_cases_imdb_rotulated_with_NO/"

rotulated_approachs = os.listdir(DIR_NAME)
try:
    rotulated_approachs.remove("summary.xlsx")
except Exception:
    pass

In [44]:
def fill_classification(row):
    if len(row["classification"]) == 1:
        if row["succeed"] == 1:
            row["classification"] += "N"
        elif row["succeed"] == 0:
            row["classification"] += "P"
    return row

In [45]:
def get_summary_row(_df, t_name):
    length = len(_df)
    passing = len(_df.query("succeed == 1"))
    failing = len(_df.query("succeed == 0"))
    fn = len(_df.query("classification == 'FN'"))
    fp = len(_df.query("classification == 'FP'"))
    vn = len(_df.query("classification == 'VN'"))
    vp = len(_df.query("classification == 'VP'"))
    fn_no = len(_df.query("classification == 'FN' and subclassification == 'NO'"))
    fn_le = len(_df.query("classification == 'FN' and subclassification == 'LE'"))
    fn_te = len(_df.query("classification == 'FN' and subclassification == 'TE'"))
    fp_no = len(_df.query("classification == 'FP' and subclassification == 'NO'"))
    fp_le = len(_df.query("classification == 'FP' and subclassification == 'LE'"))
    fp_te = len(_df.query("classification == 'FP' and subclassification == 'TE'"))
    return [t_name, length, passing, failing, fn, fp, vn, vp, fn_no, fn_le, fn_te, fp_no, fp_le, fp_te]

In [46]:
new_columns = [
    "template_name", "amostras", "passando", "falhando", "FN", "FP", "VN", "VP", "FN_NO", "FN_LE", "FN_TE", "FP_NO", "FP_LE", "FP_TE"
]

In [47]:
with pd.ExcelWriter(os.path.join(DIR_NAME, "summary.xlsx")) as writer:

    apps_summary_rows = []
    for app in sorted(rotulated_approachs):
        templates = pd.read_excel(os.path.join(DIR_NAME, app), sheet_name=None)
        app_summary_rows = []
        for t in list(templates.keys()):
            if t=="templates":
                continue
            try:
                templates[t].columns = ["text", "label", "pred", "succeed", "classification", "subclassification", "obs"]
            except ValueError as e:
                print(app, t)
                continue
            
            templates[t] = templates[t].apply(fill_classification, axis=1)

            app_summ_row = get_summary_row(templates[t], t_name=t)
            app_summary_rows.append(app_summ_row)
        app_summary_df = pd.DataFrame(data=app_summary_rows, columns=new_columns)
        app_summary_df.to_excel(writer, sheet_name=app, index=False)

        apps_summary_rows.append([app] + app_summary_df.drop(columns="template_name").sum().to_list())

    apps_columns = ["approach"] + new_columns[1:]
    apps_summary_df = pd.DataFrame(data=apps_summary_rows, columns=apps_columns).sort_values("approach")
    cols_to_perc = ["passando", "falhando", "FN", "FP", "VN","VP"]
    cols_to_perc.reverse()
    for col, loc in zip(cols_to_perc, range(len(cols_to_perc) + 2, 2, -1)):
        new_col = col + "(%)"

        total_percentage = apps_summary_df["amostras"]
        if col in ["FN", "FP", "VN","VP"]:
            if col[-1] == "N":
                total_percentage = apps_summary_df["passando"]
            elif col[-1] == "P":
                total_percentage = apps_summary_df["falhando"]
            else:
                raise "Impossible"
        
        apps_summary_df.insert(loc, new_col, (apps_summary_df[col]/total_percentage*100).map(lambda x: f"{x:.1f}"))
        

    apps_summary_df.to_excel(writer, sheet_name="general_summary", index=False)